# Lecture 2

This Document contains the simulation of examples provided in Lecture 2 of Introduction to Rienforcement Learning by David Silver

Author: Krishna Chaitanya Kosaraju

email: kkrishnachaitanya89@gmail.com

In [0]:
import numpy as np

#Part one 

State Value function for Markov Reward Process (MRP)

In [0]:
#Propability Trasition Matrix
# C1, C2, C3, Pass, Pub, FB, Sleep
#              C1 , C2,  C3, Pass, Pub, FB,  Sleep
P = np.array([[0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0],
              [0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.2],
              [0.0, 0.0, 0.0, 0.6, 0.4, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
              [0.2, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0],
              [0.1, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

# test print('Column sum of the Propability transition matrix is', np.dot(P,np.full(7,1)))
# MDP states and their Rewards
#r = np.array([-2, -2, -2, 10, 1, -1, 0])
r = {'C1' : -2,
     'C2' : -2,
     'C3' : -2,
    'Pass': 10,
     'Pub': 1,
     'FB' :-1,
     'Sleep': 0}

In [0]:
def net_reward(walk, gamma = 0.5):
  """ Return the net discounted reward for the given list of steps in 'walk',
  'walk' should be keys of r"""
  temp = 0
  for step in reversed(walk):
    temp = gamma* temp + r[step]
    #print(temp)
  return temp

walk1 = ['C1', 'C2', 'C3', 'Pass', 'Sleep'];
walk2 = ['C1', 'FB', 'FB', 'C1', 'C2', 'Sleep'];
walk3 = ['C1', 'C2', 'C3', 'Pub', 'C2', 'C3', 'Pass', 'Sleep'];
walk4 = ['FB', 'FB', 'FB', 'C1', 'C2', 'C3', 'Pub', 'C2', 'Sleep']

print("Net reward in walk 1 is", net_reward(walk1))
print("Net reward in walk 2 is", net_reward(walk2))
print("Net reward in walk 3 is", net_reward(walk3))
print("Net reward in walk 4 is", net_reward(walk4))

Net reward in walk 1 is -2.25
Net reward in walk 2 is -3.125
Net reward in walk 3 is -3.40625
Net reward in walk 4 is -2.1875


In [0]:
# Value function with 'P' as the policy

def value_fun(rewards = r, policy = P, iter = 100000, gamma = 0.9):
  R = np.array(list(r.values()))
  values = R
  for i in range(iter):
    #update rule using Bellman equation v = r + gamma*P*v
    values = R + gamma*np.dot(P,values)
  return values

print("State-Value Function with gamma = 0 is ", np.round(value_fun(gamma =0.0),2))

print("State-Value Function with gamma = 0.9 is ", np.round(value_fun(gamma =0.9),2))

print("State-Value Function with gamma = 1 is ", np.round(value_fun(gamma =1),2))


State-Value Function with gamma = 0 is  [-2. -2. -2. 10.  1. -1.  0.]
State-Value Function with gamma = 0.9 is  [-5.01  0.94  4.09 10.    1.91 -7.64  0.  ]
State-Value Function with gamma = 1 is  [-12.54   1.46   4.32  10.     0.8  -22.54   0.  ]


# Part - two 
The resuts in this section requires the knowledge of Algorithms from Lecture 3

State and Action Value Function for student Markov Decision Process
1.   There are 5 states, denote them by -- S1,  S2, S3, S4, S5.
2.   There are 6 actions in total -- Study, Facebook, Sleep,  Pub, Quit.
3.   Not every action is available to the every state
4.   Note that Reward depends on the state and action 

In [0]:
# Enivornment 
def MDP(s,a):
  # given current state and action, output an array of triplets : 
  #(next_state, propability(next_state|state), reward(state,action)) 
  mdp={'S1' : {'Study' : [('S2',1,-2)], 'Facebook': [('S4',1,-1)]},
       'S2' : {'Study' : [('S3',1,-2)], 'Sleep': [('S5',1,0)]},
       'S3' : {'Study' : [('S5',1,10)], 'Pub' : [('S1',0.2,1), ('S2',0.4,1), ('S3',0.4,1)]},
       'S4' : {'Quit' : [('S1',1,0)], 'Facebook': [('S4',1,-1)]},
       'S5': {'Sleep' : [('S5',1,0)]}}
  return mdp[s][a]

Next we use the Value Iteration to compute the Value function and the Q function 


In [0]:

#Consider the following policy
policy = {'S1' : {'Study' : 0.5, 'Facebook': 0.5},
       'S2' : {'Study' : 0.5, 'Sleep': 0.5},
       'S3' : {'Study' : 0.5, 'Pub' : 0.5},
       'S4' : {'Quit' : 0.5, 'Facebook': 0.5},
       'S5': {'Sleep' : 1}}

def Qvalue_function(policy = policy, iter_policy =500, gamma = 1):
  # initializing Q-function and V-function as a function of state and action
  V= {'S1':0,'S2':0,'S3':0,'S4':0,'S5':0}
  Q= {'S1' : {'Study' : 0.5, 'Facebook': 0.5},
       'S2' : {'Study' : 0.5, 'Sleep': 0.5},
       'S3' : {'Study' : 0.5, 'Pub' : 0.5},
       'S4' : {'Quit' : 0.5, 'Facebook': 0.5},
       'S5': {'Sleep' : 1}}
  # we will use both Q and V functions to find one another
  #V(s) = Policy(s,a) * Q(s,a)
  #Q(s,a) = R(s,a) + gamma* state_transitions(s,s') * V(s')
  for i in range(iter_policy):
    for s in ['S1','S2','S3','S4','S5']:
      # introducing a dummy variable to hold the right side of V(s)
      dummy = 0 
      for a in policy[s]:
        #we compute the second term in right hand side of equation Q(s,a) and store in the following variable
        second_term= 0#{'S1':0,'S2':0,'S3':0,'S4':0,'S5':0}
        #quiry the environment and collect the samples, i.e., next_state, P(state,next_state) and reward
        for next_state, prob, reward in MDP(s,a):
          second_term = second_term + prob * V[next_state]
        Q[s][a] = reward + gamma * second_term
        dummy = dummy + Q[s][a]*policy[s][a]
      V[s] = dummy
  return Q, V
          
Qvalue_function()


  

({'S1': {'Facebook': -3.3076923076923084, 'Study': 0.6923076923076925},
  'S2': {'Sleep': 0, 'Study': 5.384615384615385},
  'S3': {'Pub': 4.76923076923077, 'Study': 10},
  'S4': {'Facebook': -3.3076923076923084, 'Quit': -1.307692307692308},
  'S5': {'Sleep': 0}},
 {'S1': -1.307692307692308,
  'S2': 2.6923076923076925,
  'S3': 7.384615384615385,
  'S4': -2.3076923076923084,
  'S5': 0})

Next, we find the optimal policy and (state) value functions using value iteration (with k=5)

In [0]:
#Value Iteration
policy = {'S1' : {'Study' : 0.5, 'Facebook': 0.5},
       'S2' : {'Study' : 0.5, 'Sleep': 0.5},
       'S3' : {'Study' : 0.5, 'Pub' : 0.5},
       'S4' : {'Quit' : 0.5, 'Facebook': 0.5},
       'S5': {'Sleep' : 1}}
def Agent(policy = policy, iter_value = 100, iter_policy =5, gamma = 1):
  #We now act greedily with respective to the Value function
  for i in range(iter_value):
    Q, V= Qvalue_function(policy = policy, iter_policy = iter_policy, gamma = gamma)
    #We now act greedily with respective to Q
    #One can also follow episilon greedily, which is actually a better way 
    for state in ['S1','S2','S3','S4','S5']:
      opt_policy = max(Q[state], key=Q[state].get)
      policy[state][opt_policy] = 1
      not_opt_policy = min(Q[state], key=Q[state].get)
      policy[state][not_opt_policy] = 0
      #policy[state][max(Q[state], key=Q[state].get))] = 1
      #policy[state][min(Q[state], key=Q[state].get))] = 0
  return policy, Q, V


optimal_policy, optimal_Q, optimal_V = Agent()

print("Optimal Actions")
for state in ['S1','S2','S3','S4','S5']:
  print(state, max(optimal_Q[state], key=optimal_Q[state].get))
print("\n")
print("Optimal State-Action Value function Q(s,a) is")
for state in ['S1','S2','S3','S4','S5']:
  for action in  optimal_Q[state]:
    print(state, action, optimal_Q[state][action])
print("\n")
print("Optimal State Value function V(s) is")
for state in ['S1','S2','S3','S4','S5']:
  print(state, optimal_V[state])

Optimal Actions
S1 Study
S2 Study
S3 Study
S4 Quit
S5 Sleep


Optimal State-Action Value function Q(s,a) is
S1 Study 6.0
S1 Facebook 5.0
S2 Study 8.0
S2 Sleep 0
S3 Study 10
S3 Pub 9.4
S4 Quit 6.0
S4 Facebook 5.0
S5 Sleep 0


Optimal State Value function V(s) is
S1 6.0
S2 8.0
S3 10.0
S4 6.0
S5 0
